# APEX QTL association testing
This notebook implements a workflow for using the APEX to conduct analysis. APEX implements a linear mixed model for association testing. This is potentially useful for analysis with related individuals.

**FIXME: cite APEX paper**

## Input
- `--molecular-pheno`, The `bed.gz` file containing the table describing the molecular phenotype. It should have a companion index file in `tbi` format.

- `genotype_list` a list of whole genome vcf file for each chromosome, those vcf are converted beforehand from plink trio in the data_processing sections

- `grm_list` is a file containing list of grm matrixs that generated by the GRM module of this pipeline.

- `covariate` is a file with #id + samples name as colnames and each row a covariate: fixed and known covariates as well as hidden covariates recovered from factor analysis.


## Output

A sets of summary statistics files for each chromosome, for both nomial significance for each test, as well as region (gene) level association evidence.

## Command interface

In [2]:
sos run APEX.ipynb -h

usage: sos run APEX.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  LMM_null
  APEX_cis
  cis
  APEX_trans
  trans

Global Workflow Options:
  --molecular-pheno-list VAL (as path, required)
                        Path to the input molecular phenotype file, per chrm, in
                        bed.gz format.
  --covariate VAL (as path, required)
                        Covariate file, in similar format as the molecular_pheno
  --genotype-file-list VAL (as path, required)
                        Genotype file in vcf format, per chrm
  --grm-list VAL (as path, required)
                        grm file in pairwise table format , per chrm
  --cwd . (as path)
                       

## Example



In [ ]:
sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/APEX.ipynb APEX_cis   \
--genotype_file_list GRCh38_liftedover_sorted_all.add_chr.leftnorm.filtered.renamed.filtered.renamed.filtered.filtered.vcf_files_list.txt \
--molecular_pheno_list /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/data_preprocessing/ALL/phenotype_data/ALL.log2cpm.bed.processed_phenotype.per_chrom.recipe   \
--grm_list  data_preprocessing/genotype/grm/plink_files_list.loco_grm_list.txt  \
--covariate data_preprocessing/ALL/covariates/ALL.covariate.pca.BiCV.cov.gz     \
--cwd /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/Association_scan/ALL/APEX    \
--container /mnt/mfs/statgen/containers/apex.sif -J 50 -c csg.yml -q csg -n --no-LMM &

## Global parameter settings
The section outlined the parameters that can be set in the command interface.

**FIXME: please revisit this input list for various default parameter setting, in particular the list of genotypes to load. You can copy and paste the genotype load code from genotype_reformatting for now. I'll repurpose the LDtools package to include those codes in the future so we dont have to copy them over and over again.**

**FIXME: can we combine APEX_3 and APEX_4**?

In [5]:
[global]
# Path to the input molecular phenotype file, per chrm, in bed.gz format.
parameter: molecular_pheno_list = path
# Covariate file, in similar format as the molecular_pheno
parameter: covariate = path
# Genotype file in vcf format, per chrm
parameter: genotype_file_list = path
# grm file in pairwise table format , per chrm
parameter: grm_list = path("./")
# Path to the work directory of the analysis.
parameter: cwd = path('./')
# Specify the number of jobs per run.
parameter: job_size = 2
# Container option for software to run the analysis: docker or singularity
parameter: container = ''
# Prefix for the analysis output
parameter: name = 'ROSMAP'
# Specify the scanning window for the up and downstream radius to analyze around the region of interest, in units of bp
parameter: window = 1000000
# Number of threads
parameter: numThreads = 1
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Walltime 
parameter: walltime = '5h'
parameter: mem = '80G'



import pandas as pd
molecular_pheno_chr_inv = pd.read_csv(molecular_pheno_list,sep = "\t")
geno_chr_inv = pd.read_csv(genotype_file_list,sep = "\t")
input_inv = molecular_pheno_chr_inv.merge(geno_chr_inv, on = "#id")
parameter: LMM = False
if LMM:
    grm_chr_inv =  pd.read_csv(grm_list,sep = "\t")
    input_inv = input_inv.merge(grm_chr_inv,on = "#id")
input_inv = input_inv.values.tolist()

## LMM Regression  
This step are done to precompute and store:
1. LMM null models and trait residuals
2. spline terms for LMM genotypic variances to speed up downstream analysis

In [ ]:
[LMM_null]
input: for_each = "input_inv"
output: f'{cwd:a}/{path(_input_inv[1]):bnn}.theta.gz'
task: trunk_workers = 1, trunk_size=job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout', container = container
    apex lmm --vcf $[_input_inv[2]] \
    --bed $[_input_inv[1]] \
    --cov $[covariate] \
    --out $[_output[0]:nn] \
    --grm $[_input_inv[3]] \
    --get-theta 

## QTL asscoiation testing 
This step generate the cis-QTL and trans-QTL summary statistics and for downstream analysis from summary statistics. The analysis is done per chromosome to reduce running time.

In [ ]:
[APEX_cis_1, cis]
if LMM:
    sos_run("LMM_null")
input: for_each = "input_inv"
output:f'{cwd:a}/{path(_input_inv[1]):bnn}{".LMM" if LMM else ".OLS"}.cis_long_table.txt.gz',
       f'{cwd:a}/{path(_input_inv[1]):bnn}{".LMM" if LMM else ".OLS"}.cis_gene_table.txt.gz',
       f'{cwd:a}/{path(_input_inv[1]):bnn}{".LMM" if LMM else ".OLS"}.cis_sumstats.txt.gz'
parameter: theta = path(f'{cwd:a}/{path(_input_inv[1]):bnn}.theta.gz')
task: trunk_workers = 1, trunk_size=job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container
    apex cis --vcf $[_input_inv[2]] \
    --bed $[_input_inv[1]] \
    --cov $[covariate] \
    --out $[_output[0]:nnn] \
    --long $[f'--theta {theta} --grm {_input_inv[3]}'  if LMM else ""] \
    --window $[window] 

In [ ]:
[APEX_trans_1, trans]
if LMM:
    sos_run("LMM_null")
parameter: keep_gene = ""
input: for_each = "input_inv"
output: f'{cwd:a}/{path(_input_inv[1]):bnn}{".LMM" if LMM else ".OLS"}.trans_long_table.txt.gz',
        f'{cwd:a}/{path(_input_inv[1]):bnn}{".LMM" if LMM else ".OLS"}.trans_gene_table.txt.gz'
parameter: theta = path(f'{cwd:a}/{path(_input_inv[1]):bnn}.theta.gz')
task: trunk_workers = 1, trunk_size=job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container
    apex trans --vcf $[_input_inv[2]] \
    --bed $[_input_inv[1]] \
    --cov $[covariate] \
    --out $[_output[0]:nnn] \
    --gene "$[keep_gene]" \
    --long $[f'--theta {theta} --grm {_input_inv[3]}'  if LMM else ""] 

In [ ]:
[APEX_*_2]
output: f'{_input[0]:nn}.reformated.txt'
task: trunk_workers = 1, trunk_size=job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container = container
    library("dplyr")
    library("tibble")
    library("readr")
    library("purrr")
    data <- read_delim("$[_input[0]]",delim = "\t")
    data = data%>%mutate(variant_id = pmap_chr(list(a = `#chrom`,b = pos,c = ref, d = alt),function(a,b,c,d) paste(c(a,":",b,"_",c,"_",d),collapse = "")))%>%rename(chrom = `#chrom`)
    data %>%write_delim("$[_output]",delim = "\t")

In [ ]:
[APEX_*_3]
input: group_by = "all"
output: f'{cwd}/APEX_QTL_recipe.tsv',f'{cwd:a}/APEX_column_info.txt'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    import pandas as pd 
    data_tempt = pd.DataFrame({
    "#id" : [int(x.split(".")[-6].replace("chr","")) for x in  [$[_input:r,]]],
    "sumstat_dir" : [$[_input:r,]],\
    "column_info" : $[_output[1]:r]
    })
    column_info_df = pd.DataFrame( pd.Series( {"ID": "GENE,CHR,POS,A0,A1",
          "CHR": "chrom",
          "POS": "pos",
          "A0": "ref",
          "A1": "alt",
          "SNP": "variant_id",
          "STAT": "beta",
          "SE": "se",
          "P": "pval",
          "GENE": "gene"}), columns = ["APEX"] )
    data_tempt.to_csv("$[_output[0]]",index = False,sep = "\t" )
    column_info_df.to_csv("$[_output[1]]",index = True,sep = "\t" )